# Install lib

In [1]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
# use this function to print the json with the indent=4
def jprint(data):
  print(json.dumps(data,indent=4))

# Spark for processing

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, regexp_replace

# Initialize the Spark session
spark = SparkSession.builder \
    .appName("DSDE2024") \
    .getOrCreate()

# Load the CSV file
df = spark.read.option("header", "true").csv('../out_spark/before_spark/input_concatenated.csv')

# Remove brackets and single quotes, then split into array
df = df.withColumn("Class", regexp_replace(col("Class"), "[\\[\\]'']", ""))  # Remove [ ], and '
df = df.withColumn("Class", split(col("Class"), ",\s*"))  # Split into array

# Show the transformed DataFrame to verify
df.show(truncate=False)

+---------+----------+------+------------+-------------+--------+------------------------+
|id       |publicDate|source|coAuthorship|citationCount|refCount|Class                   |
+---------+----------+------+------------+-------------+--------+------------------------+
|201800282|15/11/2018|1     |3           |29           |89      |[ENER, PHYS]            |
|201800276|17/11/2018|1     |5           |11           |18      |[CHEM, BIOC, AGRI]      |
|201800044|11/12/2018|1     |5           |76           |60      |[ENVI, CENG, MATE, ENER]|
|201800249|01/12/2018|1     |4           |9            |39      |[EART, AGRI]            |
|201800043|12/12/2018|1     |2303        |59           |101     |[PHYS]                  |
|201800271|20/11/2018|1     |2           |53           |21      |[COMP]                  |
|201800285|14/11/2018|1     |1           |5            |16      |[AGRI, ENVI]            |
|201800088|01/12/2018|1     |32          |23           |107     |[PHYS, EART]            |

In [5]:
df = df.dropDuplicates(["id"])

In [6]:
df.count()

1838

In [7]:
# cast type accordingly
df = df.withColumn('citationCount', df.citationCount.cast('int'))
df = df.withColumn('coAuthorship', df.coAuthorship.cast('int'))
df = df.withColumn('refCount', df.refCount.cast('int'))

In [8]:
from pyspark.sql.functions import avg, min, max, countDistinct, explode, split, col, round, sum

max_values = df.agg(
    max("citationCount").alias("maxCitation"),
    max("refCount").alias("maxRef"),
    max("coAuthorship").alias("maxCoAuthor")
).collect()[0]

# max value for each feature for normalization
max_citation = max_values["maxCitation"]
max_ref = max_values["maxRef"]
max_coauthor = max_values["maxCoAuthor"]

In [9]:
# find all Class
genre_counts = df.withColumn("Genre", explode(col("Class")))\
                 .groupBy("Genre")\
                 .count()  

print('number of all the class:', genre_counts.count())
genre_counts.show()

number of all the class: 27
+-----+-----+
|Genre|count|
+-----+-----+
| COMP|  131|
| MATE|  141|
| IMMU|   87|
| ARTS|   20|
| PHYS|  188|
| HEAL|   16|
| PSYC|    9|
| BIOC|  202|
| NEUR|   26|
| VETE|   66|
| ENGI|  213|
| PHAR|   60|
| MEDI|  483|
| ECON|   32|
| MATH|   48|
| MULT|  299|
| ENVI|  159|
| DECI|   23|
| AGRI|  149|
| ENER|   93|
+-----+-----+
only showing top 20 rows



In [10]:
# explode the class
exploded_df = df.withColumn("Class", explode(col("Class")))
# exploded_df.show(20)

In [11]:
# drop na for invalid rows
cleaned_df = exploded_df.dropna()
print(cleaned_df.count())
cleaned_df.show(5)

2852
+---------+----------+------+------------+-------------+--------+-----+
|       id|publicDate|source|coAuthorship|citationCount|refCount|Class|
+---------+----------+------+------------+-------------+--------+-----+
|201800000|31/12/2018|     1|           2|            1|      76| MEDI|
|201800001|31/12/2018|     1|           2|            1|       4| ENGI|
|201800001|31/12/2018|     1|           2|            1|       4| MATE|
|201800002|31/12/2018|     1|           4|           21|      42| CHEM|
|201800002|31/12/2018|     1|           4|           21|      42| CENG|
+---------+----------+------+------------+-------------+--------+-----+
only showing top 5 rows



In [12]:
# compute the score for each paper
cleaned_df = cleaned_df.withColumn(
    "Score",
    round(
        col("source") * (
            0.4 * (col("citationCount") / max_citation * 10) +
            0.2 * (col("refCount") / max_ref * 10) +
            0.1 * (col("coAuthorship") / max_coauthor * 10)
        ), 4
    )
)

cleaned_df.show(10)

+---------+----------+------+------------+-------------+--------+-----+------+
|       id|publicDate|source|coAuthorship|citationCount|refCount|Class| Score|
+---------+----------+------+------------+-------------+--------+-----+------+
|201800000|31/12/2018|     1|           2|            1|      76| MEDI| 0.264|
|201800001|31/12/2018|     1|           2|            1|       4| ENGI|0.0203|
|201800001|31/12/2018|     1|           2|            1|       4| MATE|0.0203|
|201800002|31/12/2018|     1|           4|           21|      42| CHEM|0.2681|
|201800002|31/12/2018|     1|           4|           21|      42| CENG|0.2681|
|201800002|31/12/2018|     1|           4|           21|      42| ENGI|0.2681|
|201800003|31/12/2018|     1|           8|           44|      45| CHEM| 0.416|
|201800003|31/12/2018|     1|           8|           44|      45| PHYS| 0.416|
|201800003|31/12/2018|     1|           8|           44|      45| MATE| 0.416|
|201800004|31/12/2018|     1|           6|          

In [12]:
from pyspark.sql.functions import to_date, year, quarter, col, sum, round, count

# Assuming SparkSession is already created and cleaned_df is preloaded DataFrame
# Convert publicDate from string to date type
cleaned_df = cleaned_df.withColumn("publicDate", to_date(col("publicDate"), "dd/MM/yyyy"))

# Extract Year and Quarter from publicDate
cleaned_df = cleaned_df.withColumn("Year", year(col("publicDate")))
cleaned_df = cleaned_df.withColumn("Quarter", quarter(col("publicDate")))

# Group by Class, Year, and Quarter and perform aggregations
grouped_df = cleaned_df.groupBy("Class", "Year", "Quarter").agg(
    round(sum("Score"), 4).alias("Total Score"),
    count("id").alias("Paper Count")  # Count the number of papers per group
)

# Display the result
grouped_df.show()

+-----+----+-------+-----------+-----------+
|Class|Year|Quarter|Total Score|Paper Count|
+-----+----+-------+-----------+-----------+
| IMMU|2019|      3|     2.3568|          1|
| NEUR|2020|      2|     1.8807|          1|
| MEDI|2019|      4|   270.1514|        122|
| PHYS|2018|      4|    23.7444|         10|
| MEDI|2023|      3|      1.208|          2|
| PHYS|2023|      1|     7.0614|          1|
| CHEM|2021|      4|    30.0819|          9|
| MEDI|2021|      2|      4.196|          3|
| COMP|2022|      4|    32.7025|         10|
| MEDI|2023|      4|   640.2262|        117|
| PHYS|2022|      4|    34.0808|         10|
| NEUR|2021|      4|    32.2437|         10|
| COMP|2020|      4|    22.6319|          8|
| IMMU|2022|      3|     7.7949|          1|
| BIOC|2019|      4|     37.203|         18|
| MEDI|2023|      2|    11.4988|          8|
| IMMU|2023|      4|    48.8768|         10|
| BIOC|2023|      4|    87.5925|         17|
| NEUR|2023|      2|      1.808|          1|
| IMMU|201

In [13]:
# Coalesce the DataFrame to 1 partition to avoid multiple part files
grouped_df.coalesce(1).write.csv(path="./nauture.csv", mode="overwrite", header=True)